In [72]:
import os
import pandas as pd
import numpy as np
import time
from tqdm import notebook as nb
import requests
from bs4 import BeautifulSoup
import re
import os
import pickle
import glob

import matplotlib.pyplot as plt
from collections import Counter
import datetime

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler



In [109]:
players_root = "/mounted/data/players/csv/"
players_list = ["fan1704.csv","fan1710.csv","fan1804.csv","fan1810.csv","fan1610.csv"]
h = ''
data = pd.read_csv(players_root + "fan1904.csv")
data = data[['登番','1コース進入回数','1コース1着回数','1コース2着回数','1コース3着回数','2コース進入回数','2コース1着回数','2コース2着回数','2コース3着回数','3コース進入回数','3コース1着回数','3コース2着回数','3コース3着回数','4コース進入回数','4コース1着回数','4コース2着回数','4コース3着回数','5コース進入回数','5コース1着回数','5コース2着回数','5コース3着回数','6コース進入回数','6コース1着回数','6コース2着回数','6コース3着回数']]
for j,player in enumerate(players_list):
    tmp = pd.read_csv(players_root + player)
    tmp = tmp[['登番','1コース進入回数','1コース1着回数','1コース2着回数','1コース3着回数','2コース進入回数','2コース1着回数','2コース2着回数','2コース3着回数','3コース進入回数','3コース1着回数','3コース2着回数','3コース3着回数','4コース進入回数','4コース1着回数','4コース2着回数','4コース3着回数','5コース進入回数','5コース1着回数','5コース2着回数','5コース3着回数','6コース進入回数','6コース1着回数','6コース2着回数','6コース3着回数']]
    for i in range(1,7):
        tmp = tmp.rename(columns={f'{i}コース進入回数':f'{i}コース進入回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース1着回数':f'{i}コース1着回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース2着回数':f'{i}コース2着回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース3着回数':f'{i}コース3着回数_{j}'})
    data = pd.merge(data,tmp,how="left",on="登番")
    data.dropna(how='all', inplace=True)
    data = data.fillna(0)
    for i in range(1,7):
        data[f'{i}コース進入回数'] = data[f'{i}コース進入回数'] + data[f'{i}コース進入回数_{j}']
        data[f'{i}コース1着回数'] = data[f'{i}コース1着回数'] + data[f'{i}コース1着回数_{j}']
        data[f'{i}コース2着回数'] = data[f'{i}コース2着回数'] + data[f'{i}コース2着回数_{j}']
        data[f'{i}コース3着回数'] = data[f'{i}コース3着回数'] + data[f'{i}コース3着回数_{j}']
        data = data.drop(f'{i}コース進入回数_{j}', axis=1)
        data = data.drop(f'{i}コース1着回数_{j}', axis=1)
        data = data.drop(f'{i}コース2着回数_{j}', axis=1)
        data = data.drop(f'{i}コース3着回数_{j}', axis=1)
for i in range(1,7):
    data = data[data[f'{i}コース進入回数'] > 10]
for i in range(1,7):
    data[f'{i}コース1着率'] = (data[f'{i}コース1着回数'] / data[f'{i}コース進入回数']) * 100
    data[f'{i}コース2着率'] = (data[f'{i}コース2着回数'] / data[f'{i}コース進入回数']) * 100
    data[f'{i}コース3着率'] = (data[f'{i}コース3着回数'] / data[f'{i}コース進入回数']) * 100
    data = data.drop(f'{i}コース進入回数', axis=1)
    data = data.drop(f'{i}コース1着回数', axis=1)
    data = data.drop(f'{i}コース2着回数', axis=1)
    data = data.drop(f'{i}コース3着回数', axis=1)
data = data.fillna(0)
# data['1コース進入回数'] = data['1コース進入回数'] + data['1コース進入回数_0']
data
# print(players_list)

,登番,1コース1着率,1コース2着率,1コース3着率,2コース1着率,2コース2着率,2コース3着率,3コース1着率,3コース2着率,3コース3着率,4コース1着率,4コース2着率,4コース3着率,5コース1着率,5コース2着率,5コース3着率,6コース1着率,6コース2着率,6コース3着率
0,2014.0,21.649485,15.463918,25.773196,11.650485,24.271845,25.242718,8.571429,11.428571,21.904762,8.163265,17.346939,32.653061,7.207207,14.414414,29.729730,2.150538,8.602151,13.978495
1,2538.0,22.471910,20.224719,11.235955,4.794521,18.493151,21.232877,7.377049,13.114754,21.311475,0.990099,8.910891,17.821782,1.492537,7.462687,14.925373,0.000000,2.985075,13.432836
2,2787.0,38.383838,28.282828,11.111111,19.000000,24.000000,23.000000,9.090909,19.834711,19.008264,6.086957,9.565217,26.956522,0.000000,10.309278,18.556701,0.000000,1.639344,9.836066
3,2841.0,22.105263,35.789474,18.947368,6.504065,25.203252,21.138211,6.666667,22.857143,20.000000,9.600000,14.400000,29.600000,2.631579,8.771930,20.175439,0.000000,5.357143,7.142857
4,2844.0,28.571429,19.780220,12.087912,16.037736,26.415094,20.754717,5.357143,11.607143,19.642857,7.766990,15.533981,16.504854,1.000000,3.000000,9.000000,1.785714,3.571429,7.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,4960.0,52.500000,22.500000,15.000000,14.285714,19.047619,28.571429,16.326531,16.326531,18.367347,6.557377,16.393443,26.229508,6.593407,10.989011,20.879121,2.209945,8.287293,8.839779
1476,4964.0,44.444444,11.111111,16.666667,10.526316,26.315789,15.789474,15.000000,15.000000,5.000000,10.769231,16.923077,15.384615,8.653846,13.461538,16.346154,1.463415,5.365854,6.341463
1490,4979.0,15.384615,23.076923,15.384615,6.250000,0.000000,25.000000,0.000000,12.000000,24.000000,0.000000,0.000000,6.896552,1.587302,4.761905,14.285714,2.721088,2.721088,8.163265
1493,4982.0,18.181818,27.272727,0.000000,5.882353,23.529412,5.882353,0.000000,12.500000,12.500000,0.000000,0.000000,18.181818,0.000000,2.500000,10.000000,0.602410,0.602410,1.204819


In [110]:
file_list = sorted(glob.glob('/mounted/data/results/csv/k200*.csv'))
cols = ['1号艇着順','1号艇登番','1号艇1着率','1号艇2着率','1号艇3着率','2号艇着順','2号艇登番','2号艇1着率','2号艇2着率','2号艇3着率','3号艇着順','3号艇登番','3号艇1着率','3号艇2着率','3号艇3着率','4号艇着順','4号艇登番','4号艇1着率','4号艇2着率','4号艇3着率','5号艇着順','5号艇登番','5号艇1着率','5号艇2着率','5号艇3着率','6号艇着順','6号艇登番','6号艇1着率','6号艇2着率','6号艇3着率']
dataset = pd.DataFrame(columns=cols)
for file in file_list:
    merged = pd.DataFrame()
    results = pd.read_csv(file)
    results = pd.merge(results, data, how="left", on = "登番")
    results.isnull().sum()
    merged = merged.reset_index(drop=True)
    for i in range(0,6):
        merged.insert(0,f'{6-i}号艇3着率',results[results['艇']==6-i][f'{6-i}コース3着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇2着率',results[results['艇']==6-i][f'{6-i}コース2着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇1着率',results[results['艇']==6-i][f'{6-i}コース1着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇登番',results[results['艇']==6-i]['登番'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇着順',results[results['艇']==6-i]['着'].reset_index(drop=True))
    merged.insert(0,'1着',results[results['着']=='01']['艇'].reset_index(drop=True))
    # merged.insert(0,'2着',results[results['着']=='02']['艇'].reset_index(drop=True))
    # merged.insert(0,'3着',results[results['着']=='03']['艇'].reset_index(drop=True))
    # merged['123着'] = merged['1着'].astype(int).astype(str) + merged['2着'].astype(int).astype(str) + merged['3着'].astype(int).astype(str)
    dataset = pd.concat([dataset,merged])
dataset = dataset.reset_index(drop=True)
# dataset
# dataset.to_csv('/mounted/dataset/dataset01/dataset01.csv', index=False)
#  & (dataset['2号艇1着率'] < 20) & (dataset['3号艇1着率'] < 20) & (dataset['4号艇1着率'] < 20) & (dataset['5号艇1着率'] < 20)

In [111]:
def mean_norm(df_input):
    return df_input.apply(lambda x: (x - x.mean()) / x.std(), axis=0)

In [112]:
# dataset1 = dataset[(dataset['1号艇1着率'] > 60) & (dataset['2号艇1着率'] < 10) & (dataset['3号艇1着率'] < 10) & (dataset['4号艇1着率'] < 10) & (dataset['5号艇1着率'] < 10)]
# # dataset1['2号艇2着率'].hist(bins = 12)
# # dataset1 = dataset1[(dataset1['2号艇2着率'] > 25)]
# # dataset1['6号艇2着率'].hist(bins = 20)
# dataset1 = dataset1[dataset1['123着'] == '156']
# print(dataset['123着'].value_counts())
conditions = [
    (dataset['1号艇着順'] == "01") & (dataset["2号艇着順"] == "02") & (dataset["3号艇着順"] == "03")
]
choices = [1]
dataset['123'] = np.select(conditions, choices, default=0)
dataset.dropna(how='any',inplace=True)
dataset['1着'] = dataset['1着'] - 1
dataset
# dataset.describe()
# print(dataset1['3号艇着順'].value_counts())
# print(dataset1['4号艇着順'].value_counts())
# print(dataset1['5号艇着順'].value_counts())
# print(dataset1['6号艇着順'].value_counts())

,1号艇着順,1号艇登番,1号艇1着率,1号艇2着率,1号艇3着率,2号艇着順,2号艇登番,2号艇1着率,2号艇2着率,2号艇3着率,...,5号艇1着率,5号艇2着率,5号艇3着率,6号艇着順,6号艇登番,6号艇1着率,6号艇2着率,6号艇3着率,1着,123
1,02,4005,26.760563,16.901408,14.084507,04,3843,10.791367,25.179856,16.546763,...,1.904762,8.571429,13.333333,05,3151,1.515152,3.030303,15.151515,2.0,0
2,01,3501,59.842520,12.598425,9.448819,05,3964,5.607477,16.822430,22.429907,...,5.217391,16.521739,17.391304,06,4603,0.000000,7.207207,12.612613,0.0,0
3,01,3527,62.500000,17.708333,8.333333,04,3936,13.274336,20.353982,17.699115,...,0.952381,11.428571,19.047619,02,4041,1.470588,2.941176,19.117647,0.0,0
6,01,4259,71.428571,15.934066,5.494505,02,3773,17.910448,17.164179,18.656716,...,3.092784,5.154639,17.525773,06,3151,1.515152,3.030303,15.151515,0.0,1
8,01,4855,50.000000,13.888889,5.555556,06,4005,13.043478,16.304348,25.000000,...,2.500000,7.500000,24.166667,05,3843,1.052632,8.421053,14.736842,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41179,03,3790,49.264706,15.441176,10.294118,05,4389,10.377358,33.962264,17.924528,...,4.761905,20.634921,13.492063,04,3860,0.000000,5.825243,21.359223,2.0,0
41181,01,4266,76.582278,10.126582,3.164557,05,4319,23.076923,30.769231,10.000000,...,8.870968,15.322581,16.935484,04,4779,3.125000,3.125000,9.375000,0.0,0
41182,01,4075,72.727273,8.585859,7.575758,04,3920,26.857143,37.142857,11.428571,...,8.163265,27.551020,16.326531,06,4728,0.980392,4.901961,12.745098,0.0,0
41183,01,4468,68.503937,10.236220,5.511811,03,4907,12.820513,20.512821,12.820513,...,2.400000,14.400000,29.600000,04,3889,2.000000,4.000000,12.000000,0.0,0


In [115]:
print(dataset.dtypes)


1号艇着順      object
1号艇登番      object
1号艇1着率    float64
1号艇2着率    float64
1号艇3着率    float64
2号艇着順      object
2号艇登番      object
2号艇1着率    float64
2号艇2着率    float64
2号艇3着率    float64
3号艇着順      object
3号艇登番      object
3号艇1着率    float64
3号艇2着率    float64
3号艇3着率    float64
4号艇着順      object
4号艇登番      object
4号艇1着率    float64
4号艇2着率    float64
4号艇3着率    float64
5号艇着順      object
5号艇登番      object
5号艇1着率    float64
5号艇2着率    float64
5号艇3着率    float64
6号艇着順      object
6号艇登番      object
6号艇1着率    float64
6号艇2着率    float64
6号艇3着率    float64
1着        float64
123         int64
dtype: object


In [116]:

race_features = dataset[['1号艇1着率','1号艇2着率','1号艇3着率','2号艇1着率','2号艇2着率','2号艇3着率','3号艇1着率','3号艇2着率','3号艇3着率','4号艇1着率','4号艇2着率','4号艇3着率','5号艇1着率','5号艇2着率','5号艇3着率','6号艇1着率','6号艇2着率','6号艇3着率']]
race_features = mean_norm(race_features)
race_label = dataset['123'].astype(int)
features_train,features_test,label_train,label_test = train_test_split(race_features,race_label,test_size =0.2, random_state=1)
print(race_features.dtypes)

1号艇1着率    float64
1号艇2着率    float64
1号艇3着率    float64
2号艇1着率    float64
2号艇2着率    float64
2号艇3着率    float64
3号艇1着率    float64
3号艇2着率    float64
3号艇3着率    float64
4号艇1着率    float64
4号艇2着率    float64
4号艇3着率    float64
5号艇1着率    float64
5号艇2着率    float64
5号艇3着率    float64
6号艇1着率    float64
6号艇2着率    float64
6号艇3着率    float64
dtype: object


In [62]:
Linsvc = svm.LinearSVC(random_state=0, max_iter=3500)
Linsvc.fit(features_train,label_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC(max_iter=3500, random_state=0)

In [63]:
label_pred_Linsvc = Linsvc.predict(features_test)
print(label_pred_Linsvc)
print('confusion matrix = \n', confusion_matrix(y_true = label_test, y_pred = label_pred_Linsvc))
print('accuracy = ', Linsvc.score(features_test,label_test))

[0 0 0 ... 0 0 0]
confusion matrix = 
 [[4366    0]
 [ 361    0]]
accuracy =  0.9236302094351597


In [73]:
Kneighbor = KNeighborsClassifier(n_neighbors=1)
 
#K近傍法でデータを学習
Kneighbor.fit(features_train,label_train)
 
#K近傍法でアヤメの種類を予測
# label_pred_KNeighbor = Kneighbor.predict(features_test)
# print(label_pred_KNeighbor)

print('confusion matrix = \n', confusion_matrix(y_true = label_test, y_pred = label_pred_KNeighbor))
print('accuracy = ', Kneighbor.score(features_test,label_test))

confusion matrix = 
 [[4038  328]
 [ 332   29]]


AttributeError: 'Flags' object has no attribute 'c_contiguous'

In [60]:
#ロジスティック回帰のインスタンスを作成
LogReg = LogisticRegression(random_state=0)
#ロジスティック回帰でデータを学習
LogReg.fit(features_train,label_train)
#ロジスティック回帰でアヤメの種類を予測
label_pred_LogReg = LogReg.predict(features_test)
print(label_pred_LogReg)

print('confusion matrix = \n', confusion_matrix(y_true = label_test, y_pred = label_pred_LogReg))
print('accuracy = ', LogReg.score(features_test,label_test))

[0 0 0 ... 0 0 0]
confusion matrix = 
 [[4366    0]
 [ 361    0]]
accuracy =  0.9236302094351597


In [117]:
model = LogisticRegression(penalty='l2',          # 正則化項(L1正則化 or L2正則化が選択可能)
                           dual=False,            # Dual or primal
                           tol=0.00001,            # 計算を停止するための基準値
                           C=1.0,                 # 正則化の強さ
                           fit_intercept=True,    # バイアス項の計算要否
                           intercept_scaling=1,   # solver=‘liblinear’の際に有効なスケーリング基準値
                           class_weight=None,     # クラスに付与された重み
                           random_state=None,     # 乱数シード
                           solver='lbfgs',        # ハイパーパラメータ探索アルゴリズム
                           max_iter=1000,          # 最大イテレーション数
                           multi_class='auto',    # クラスラベルの分類問題（2値問題の場合'auto'を指定）
                           verbose=0,             # liblinearおよびlbfgsがsolverに指定されている場合、冗長性のためにverboseを任意の正の数に設定
                           warm_start=False,      # Trueの場合、モデル学習の初期化に前の呼出情報を利用
                           n_jobs=None,           # 学習時に並列して動かすスレッドの数
                           l1_ratio=None          # L1/L2正則化比率(penaltyでElastic Netを指定した場合のみ)
                          )

model.fit(features_train,label_train)

LogisticRegression(max_iter=1000, tol=1e-05)

In [118]:
df_model = pd.DataFrame(index=['1号艇1着率','1号艇2着率','1号艇3着率','2号艇1着率','2号艇2着率','2号艇3着率','3号艇1着率','3号艇2着率','3号艇3着率','4号艇1着率','4号艇2着率','4号艇3着率','5号艇1着率','5号艇2着率','5号艇3着率','6号艇1着率','6号艇2着率','6号艇3着率'])
df_model["偏回帰係数"] = model.coef_[0]
df_model

,偏回帰係数
1号艇1着率,0.397388
1号艇2着率,-0.003873
1号艇3着率,0.036497
2号艇1着率,0.213917
2号艇2着率,0.176935
2号艇3着率,0.093199
3号艇1着率,0.043926
3号艇2着率,0.125111
3号艇3着率,0.113299
4号艇1着率,-0.148163


In [120]:
pred_proba = model.predict_proba(features_test)
label_test = label_test.reset_index(drop=True)
df_proba = pd.DataFrame()
df_proba["123"] = pred_proba[:,1]
df_proba["正解"] = label_test
df_proba[(df_proba["123"]>0.11)].shape

(789, 2)